In [1]:
!pip install deep_hedging

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached

In [2]:
import datetime as dt

import numpy as np

from deep_hedging import Frequency, ConstantRateCurve
from deep_hedging.linear.commodity_bond import CommodityBond

yfinance: pandas_datareader support is deprecated & semi-broken so will be removed in a future verison. Just use yfinance.


## Input parameters.

In [3]:
RF_RATE = 0.15
Z_SPREAD = 0.05
CONVENIENCE_YIELD = 0.02

TIME_TILL_MATURITY = 2.0
FREQUENCY = Frequency.ANNUALLY

NOTIONAL = 5_000

## Create curves.

In [4]:
yield_curve = ConstantRateCurve(
    currency="RUB",
    constant_rate=RF_RATE + Z_SPREAD,
    compounding_frequency=Frequency.ANNUALLY,
)
forward_curve = ConstantRateCurve(
    currency="RUB",
    constant_rate=RF_RATE - CONVENIENCE_YIELD,
    compounding_frequency=Frequency.ANNUALLY,
)

## Create bond.

### Check the composition of the bond.

In [5]:
start = dt.datetime.today()
end = start + dt.timedelta(days=int(round(TIME_TILL_MATURITY * 365)))

gold_bond = (
    CommodityBond(
        yield_curve=yield_curve,
        start_date=start,
        end_date=end,
        frequency=FREQUENCY,
        yield_curve_commodity=forward_curve,
    )
    * NOTIONAL
)
gold_bond

StructuredNote of:
1. LONG 6780.0 units of ZeroCouponBond:
* CCY = Currency.RUB
* Term = 2.0 years
* YTM = 20.0%
* Start Date = 2024-07-23 19:42:29.885673
* End Date = 2026-07-23 19:42:29.885673.

2. LONG 5309.7345 units of Forward:
* Term = 2.0 years
* Strike = 127.69%
* Start Date = 2024-07-23 19:42:29.885673
* End Date = 2026-07-23 19:42:29.885673.

3. LONG 350.0 units of ZeroCouponBond:
* CCY = Currency.RUB
* Term = 1.0 years
* YTM = 20.0%
* Start Date = 2024-07-23 19:42:29.885673
* End Date = 2025-07-24 00:00:00.

4. LONG 309.7345 units of Forward:
* Term = 1.0 years
* Strike = 113.0%
* Start Date = 2024-07-23 19:42:29.885673
* End Date = 2025-07-24 00:00:00.


### Check the payment schedule.
The schedule accounts for business days and holidays.

In [6]:
gold_bond.schedule

[Timestamp('2024-07-24 00:00:00'),
 Timestamp('2025-07-24 00:00:00'),
 Timestamp('2026-07-24 00:00:00')]

### You can amend the schedule, if needed.

In [7]:
new_schedule = gold_bond.schedule
new_schedule[1] = gold_bond.schedule[1] + dt.timedelta(days=5)
gold_bond.substitute_schedule(new_schedule)
gold_bond.schedule

[Timestamp('2024-07-24 00:00:00'),
 Timestamp('2025-07-29 00:00:00'),
 Timestamp('2026-07-24 00:00:00')]

## Let's check the coupon that the client will receive.

In [8]:
print(f"Coupon, {gold_bond.fixed_coupon * 100:.4f}% of gold fixing.")

Coupon, 6.1973% of gold fixing.


In [9]:
print(f"Price at inception = {gold_bond.price() * 100 / NOTIONAL:.2f}%")

Price at inception = 100.00%


## You can get the payments by passings the spot fixings.

In [10]:
SPOT_FIXINGS = np.array([NOTIONAL, 7_000, 7_500])
payments = gold_bond.payments(SPOT_FIXINGS)
payments

array([ 433.81219595, 7964.79878138])

## Check additionally that the final payment is equal to the structured portfolio's payoff.

In [11]:
assert np.allclose(payments[-1], SPOT_FIXINGS[-1] * (1 + gold_bond.fixed_coupon), atol=1e-4)